# AGREGACION DE DATOS DESDE EL DATASET DEL PROGRAMA DE INDICADORES DE GESTIÓN DE ORGANISMOS OPERADORES

**Descripcion**

En este documento se describen las consideraciones que se tomaron para la agregación de datos desde el dataset proporcionado por el Programa de indicadores de Gestion de Organismos Operadores para su uso en la Plataforma de Conocimiento de Ciudades   Sustentables. 

**Siglas**

- **INEGI**: Instituto Nacional de Geografía y Estadística.
- **PCCS**: Plataforma de Conocimiento de Ciudades Sustentables.
- **SUN**: Sistema Urbano Nacional
- **PIGOO**: Programa de Indicadores del Gestión de Organismos Operadores

**Repositorio de datasets**

Dataset Pigoo: 
https://github.com/INECC-PCCS/01_Dmine/blob/master/Datasets/Pigoo/CiudadesPIGOO_ClaveInegi.xlsx

Dataset Ciudades que forman parte del subsistema principal del SUN: 
https://github.com/INECC-PCCS/01_Dmine/blob/master/00_Generales/sun_main.csv

In [45]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib

module_path = r'D:\PCCS\01_Dmine\Scripts'
if module_path not in sys.path:
    sys.path.append(module_path)
from SUN.asignar_sun import asignar_sun
from SUN_integridad.SUN_integridad import SUN_integridad
from SUN.CargaSunPrincipal import getsun

In [46]:
# Configuracion del sistema
import sys; print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


## 1. Revisión y estandarización inicial al DataSet Pigoo

El 2 de octubre de 2017 se realizó por medio de correo electrónico, una solicitud de aclaracion al PIGOO del nivel de desagregación de los datos disponibles en su página (Ver archivo Aclaracion_desagregacion.pdf).

En respuesta a esta solicitud, el PIGOO proporcionó un archivo de excel (**CiudadesPIGOO_ClaveInegi.xlsx**) que incluye una clasificación de las ciudades del dataset PIGOO por clave geoestadística

In [47]:
# Carga del dataset "CiudadesPIGOO_ClaveInegi.xlsx" al sistema
pigoo_inegi = r'D:\PCCS\01_Dmine\Datasets\Pigoo\CiudadesPIGOO_ClaveInegi.xlsx'
pigoo_inegi_df = pd.read_excel(pigoo_inegi, sheetname='OOAPAS-PIGOO', index_col=0,
                      dtype={'Clave-Estado-Inegi': str,
                             'Clave-Municipio-Inegi': str,
                             'Clave-Localidad-Inegi': str})
pigoo_inegi_df.head()

,Nombre- PIGOO,Organismo Operador,Siglas,Estado,Municipio-Inegi,Loc-Inegi,Clave-Estado-Inegi,Clave-Municipio-Inegi,Clave-Localidad-Inegi
N°,,,,,,,,,
1,Aguascalientes,Comisión Ciudadana de Agua Potable y Alcantari...,CCAPAMA,Aguascalientes,Aguascalientes,NaN,01,001,nan
2,Jesús María,Comisión de Agua Potable Alcantarillado y Sane...,CAPAS,Aguascalientes,Jesús María,NaN,01,005,nan
3,Calvillo,Organismo Operador de Servicios de Agua de Cal...,OOSAC,Aguascalientes,Calvillo,NaN,01,003,nan
4,Asientos,Sistema de Agua Potable y Alcantarillado de As...,SAPA,Aguascalientes,Asientos,NaN,01,002,nan
5,Tecate,Comisión Estatal de Servicios Públicos de Teca...,CESPTE,Baja California,Tecate,NaN,02,003,nan


Gracias a que este dataset ya contiene etiquetas con claves geoestadísticas, es posible clasificarlas a su vez dentro de acuerdo con el Sistema Urbano Nacional. Para hacer la clasificación, se utiliza un algoritmo elaborado previamente que identifica la clave geoestadística municipal de 5 dígitos ("CVE_MUN") en cada renglón y la clasifica el municipio identificado asignándole la clave del Sistema Urbano Nacional ("CVE_SUN") a la que pertenece.
Es importante señalar las limitaciones de este algoritmo:
1. El algoritmo, además de clasificar los municipios, les asigna el nombre estándar como aparece en el marco geoestadístico del INEGI y el nombre de las ciudades como aparece en el Sistema Urbano Nacional.
2. Unicamente clasifica municipios que forma parte del Subsistema Principal del SUN. Desecha aquellos que no forman parte del Subsistema Principal.
3. Unicamente clasifica renglones cuya clave geoestadística municipal es de 5 dígitos ("CVE_MUN"). Esto es relevante pues en el dataset de entrada, la "Ciudad de México"  únicamente cuentan con 2 dígitos de clave geoestadística

In [48]:
# Estandarizacion de clave geoestadistica municipal a 5 dígitos
pigoo_inegi_df['CVE_MUN'] = pigoo_inegi_df['Clave-Estado-Inegi'].map(str) + pigoo_inegi_df['Clave-Municipio-Inegi']
# Clasificación de acuerdo al Sistema Urbano Nacional
variables_SUN = ['CVE_MUN', 'NOM_MUN', 'CVE_SUN', 'NOM_SUN', 'TIPO_SUN', 'NOM_ENT']
pigoo_sun = asignar_sun(pigoo_inegi_df, vars=variables_SUN)
Columnas = ['Nombre- PIGOO', 'CVE_MUN', 'NOM_MUN', 'CVE_SUN', 'NOM_SUN', 'TIPO_SUN', 'NOM_ENT', 'Organismo Operador', 'Siglas', 'Loc-Inegi']
pigoo_sun[Columnas].head()

,Nombre- PIGOO,CVE_MUN,NOM_MUN,CVE_SUN,NOM_SUN,TIPO_SUN,NOM_ENT,Organismo Operador,Siglas,Loc-Inegi
0,Aguascalientes,01001,Aguascalientes,001,Aguascalientes,1,Aguascalientes,Comisión Ciudadana de Agua Potable y Alcantari...,CCAPAMA,NaN
1,Jesús María,01005,Jesús María,001,Aguascalientes,1,Aguascalientes,Comisión de Agua Potable Alcantarillado y Sane...,CAPAS,NaN
2,Tecate,02003,Tecate,002,Tijuana,1,Baja California,Comisión Estatal de Servicios Públicos de Teca...,CESPTE,NaN
3,Tijuana,02004,Tijuana,002,Tijuana,1,Baja California,Comisión Estatal de Servicios Públicos de Tiju...,CESPT,NaN
4,Ensenada,02001,Ensenada,060,Ensenada,2,Baja California,Comisión Estatal de Servicios Públicos de Ense...,CESPE,NaN


## 2. Casos Especiales

En el Sistema Urbano Nacional, las Zonas Metropolitanas del Valle de México y Tampico están compuestas por municipios de diferentes estados. En el dataset PIGOO, ambas ciudades son motivo de incertidumbre por la manera en la que están etiquetadas. 
La pregunta para ambos casos es: ¿Se tiene información suficientemente para considerar un alto grado de integridad en la información?

La etiqueta con la que aparece cada una de estas ciudades en el dataset PIGOO da a entender que podrían contener la información de varios municipios en un solo renglón. Si este fuera el caso, haría falta verificar que el resto de los municipios que forman la ciudad también se encuentren dentro del dataset PIGOO.

Después de analizar ambos casos, se llegó a la conclusión de que no existen datos suficientes para ninguna de las dos ciudades.

**2.1. Valle de México**

La Zona Metropolitana "Valle de México" es un caso que requiere consideraciones particulares para su agregación a los datasets que se incluirán en la PCCS. Es necesario hacer una distinción entre la entidad llamada "Ciudad de México" y la ciudad llamada "Valle de México". Las delegaciones que componen la Ciudad De México forman parte de la Ciudad Valle de México, que además está compuesta por municipios del Estado de México, Ciudad de México e Hidalgo.

La Ciudad de México (Antes Distrito Federal) está dividida en 16 delegaciones, cada una identificada por el INEGI con una clave geoestadística.

In [67]:
sun = getsun().set_index('CVE_SUN')
cdmx = sun[sun['CVE_ENT'] == '09']
datos_cdmx = ['CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN']
cdmx[datos_cdmx]

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
CVE_SUN,,,,
013,09,Ciudad de México,09002,Azcapotzalco
013,09,Ciudad de México,09003,Coyoacán
013,09,Ciudad de México,09004,Cuajimalpa de Morelos
013,09,Ciudad de México,09005,Gustavo A. Madero
013,09,Ciudad de México,09006,Iztacalco
013,09,Ciudad de México,09007,Iztapalapa
013,09,Ciudad de México,09008,La Magdalena Contreras
013,09,Ciudad de México,09009,Milpa Alta
013,09,Ciudad de México,09010,Álvaro Obregón


En el Dataset de PIGOO la Ciudad de México está considerada como una unidad monolítica, manejada por el Sistema de Aguas de La Ciudad de México.

In [50]:
pigoo_inegi_df.loc[42]

Nombre- PIGOO                                            Ciudad De México
Organismo Operador       Sistema de Aguas de La Ciudad de México (SACMEX)
Siglas                                                               SACM
Estado                                                   Distrito Federal
Municipio-Inegi                                          Ciudad de México
Loc-Inegi                                                             NaN
Clave-Estado-Inegi                                                     09
Clave-Municipio-Inegi                                                 nan
Clave-Localidad-Inegi                                                 nan
CVE_MUN                                                             09nan
Name: 42, dtype: object

En el SUN, el Valle de México se compone de 76 municipios distribuidos en 3 entidades: 16 de la Ciudad de México, 59 del Estado de México y 1 de Hidalgo.

In [68]:
datos_sun = ['NOM_SUN', 'CVE_ENT','CVE_MUN', 'NOM_MUN']
vdmx = sun[sun.index == '013']
vdmx[datos_sun]

,NOM_SUN,CVE_ENT,CVE_MUN,NOM_MUN
CVE_SUN,,,,
013,Valle de México,09,09002,Azcapotzalco
013,Valle de México,09,09003,Coyoacán
013,Valle de México,09,09004,Cuajimalpa de Morelos
013,Valle de México,09,09005,Gustavo A. Madero
013,Valle de México,09,09006,Iztacalco
013,Valle de México,09,09007,Iztapalapa
013,Valle de México,09,09008,La Magdalena Contreras
013,Valle de México,09,09009,Milpa Alta
013,Valle de México,09,09010,Álvaro Obregón


Si la ciudad que en el dataset PIGOO aparece como Ciudad de México se considera como un resumen de las 16 delegaciones anteriormente mencionadas, sería necesario verificar que existen datos para todos los municipios restantes existen en el dataset PIGOO. No es el caso, de 60 municipios que integran la Zona Metropolitana del Valle de México fuera de la Ciudad de México (Antes Distrito Federal), el dataset PIGOO únicamente contiene 15:

In [86]:
sun_vdmx = vdmx['CVE_MUN'].tolist()    # Lista de municipios en el SUN que componen el Valle de México (vdmx)
pigoo_muns = pigoo_inegi_df['CVE_MUN'].tolist()   # Lista de municipios en el dataset Pigoo
vdmx_en_PIGOO = [i for i in pigoo_muns if i in sun_vdmx]   # Lista de municipios del vdmx en el dataset PIGOO

In [75]:
sun_vdmx = vdmx['CVE_MUN'].tolist()
pigoo_muns = pigoo_inegi_df['CVE_MUN'].tolist()
pigoo_muns

['01001',
 '01005',
 '01003',
 '01002',
 '02003',
 '02004',
 '02001',
 '02002',
 '03003',
 '03009',
 '03002',
 '03008',
 '04002',
 '04003',
 '07101',
 '07089',
 '07078',
 '07019',
 '07065',
 '08037',
 '08019',
 '08032',
 '08017',
 '08021',
 '08050',
 '08062',
 '08011',
 '08045',
 '08027',
 '05035',
 '05030',
 '05025',
 '05002',
 '05028',
 '05009',
 '05018',
 '05017',
 '05033',
 '05024',
 '06007',
 '06002',
 '09nan',
 '10007',
 '10005',
 '10023',
 '11017',
 '11007',
 '11020',
 '11027',
 '11015',
 '11034',
 '11040',
 '11043',
 '11046',
 '11001',
 '11008',
 '11016',
 '11022',
 '11023',
 '11029',
 '11021',
 '11033',
 '11037',
 '11042',
 '11035',
 '11039',
 '11014',
 '11003',
 '11002',
 '11004',
 '11005',
 '11011',
 '11018',
 '11025',
 '11026',
 '11028',
 '11031',
 '11030',
 '11032',
 '11041',
 '11044',
 '11036',
 '11019',
 '11009',
 '11010',
 '11038',
 '11013',
 '11012',
 '12001',
 '12038',
 '12035',
 '12055',
 '12029',
 '13063',
 '13048',
 '13030',
 '13024',
 '13028',
 '13077',
 '14039',


In [84]:
len()

15

In [82]:
pigoo_inegi_df[pigoo_inegi_df['CVE_MUN'] == '15109']

,Nombre- PIGOO,Organismo Operador,Siglas,Estado,Municipio-Inegi,Loc-Inegi,Clave-Estado-Inegi,Clave-Municipio-Inegi,Clave-Localidad-Inegi,CVE_MUN
N°,,,,,,,,,,
110,Tlalnepantla,Organismo Público Descentralizado Para Prestac...,OPDM,México,Tlalnepantla de Baz,NaN,15,104,nan,15104


**2.2. Tampico y Madero**

En el dataset PIGOO se identifica a "Tampico y Madero" como una sola ciudad a la que le fue asignada la clave 28038 correspondiente al municipio de Tampico.

In [60]:
datos_pigoo = ['Nombre- PIGOO', 'Organismo Operador', 'Estado', 'CVE_MUN', 'Municipio-Inegi']
pigoo_inegi_df[pigoo_inegi_df['Nombre- PIGOO'] == "Tampico Y Madero"][datos_pigoo]

,Nombre- PIGOO,Organismo Operador,Estado,CVE_MUN,Municipio-Inegi
N°,,,,,
209,Tampico Y Madero,Comisión Municipal de Agua Potable y Alcantari...,Tamaulipas,28038,Tampico


Mientras que para el SUN, el municipio 28038 - Tampico, forma parte de la Zona Metropolitana 042 - "Tampico", que está compuesta por 5 municipios incluyento 28009 - Ciudad Madero 

In [64]:
tamp = sun[sun.index == '042']
tamp[datos_sun]

,CVE_ENT,CVE_MUN,NOM_MUN,NOM_SUN
CVE_SUN,,,,
042,28,28003,Altamira,Tampico
042,28,28009,Ciudad Madero,Tampico
042,28,28038,Tampico,Tampico
042,30,30123,Pánuco,Tampico
042,30,30133,Pueblo Viejo,Tampico


Ll hecho de que esta ciudad aparezca etiquetada como *"Francisco y Madero"* crea confusión respecto a cuál es la informacion con la que se construyó este indicador. La pregunta es ¿Valdría la pena tener consideraciones especiales para esta ciudad?

## 4. Estandarizacion del dataset PIGOO

In [7]:
# Descarga del dataset
dirdestino = r'D:\PCCS\00_RawData\01_CSV'
archivodestino = r'pigoo.csv'
destino = r'{}\{}'.format(dirdestino, archivodestino)
fuente = r'http://www.pigoo.gob.mx/dashboard/exportar/excel/exportar_consulta_excel.php?tipo=max'
print('Descargando dataset ... ... ... ...')
urllib.request.urlretrieve(fuente, destino)
print('dataset descargado en {}'.format(destino))
dataset = pd.read_csv(destino,
                        header = 3)
dataset = dataset.set_index('ciudad')
dataset['ciudad'] = dataset.index

Descargando dataset ... ... ... ...
dataset descargado en D:\PCCS\00_RawData\01_CSV\pigoo.csv


Después de la solicitud de aclaración de desagregación que se hizo al pigoo, nos mandaron un dataset en donde nuevamente los nombres de las ciudades vienen cambiados. Se realizó un dataset para definir con certeza las correspondencias entre los nombres de ciudades en el dataset disponible en la pagina de PIGOO, los nombres del archivo de aclaración de desagregación y la Clave Municipal de 5 dígitos del marco geoestadístico de INEGI.

In [27]:
dataset.head()

,indicador,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
ciudad,,,,,,,,,,,,,,,
"Tecate, Baja California",Tomas con servicio continuo (%),91.99,91.46,90.40,90.68,90.14,90.43,89.82,95.96,96.04,96.23,99.39,99.44,99.12,99.23
"Tijuana, Baja California",Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
"Ensenada, Baja California",Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,78.13,98.83,98.77,99.00,97.79,97.71,97.54,97.50,NaN
"Mexicali, Baja California",Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
"Torreón, Coahuila",Tomas con servicio continuo (%),99.69,99.77,99.76,99.71,99.72,99.70,99.71,92.91,100.00,100.00,100.00,99.71,100.00,100.00


In [33]:
nom_estandar = pd.read_excel(r'D:\PCCS\01_Dmine\Datasets\Pigoo\standard-pigoo_ooapas_geo.xlsx',
                            dtype={'CVE_MUN': str})
nom_estandar.head()

,Nombre en CiudadesPIGOO_ClaveInegi,CVE_MUN,Nombre en Dataset PIGOO
0,Tecate,02003,"Tecate, Baja California"
1,Tijuana,02004,"Tijuana, Baja California"
2,Ensenada,02001,"Ensenada, Baja California"
3,Mexicali,02002,"Mexicali, Baja California"
4,Torreón,05035,"Torreón, Coahuila"


Utilizando este dataset, ya es posible asignar claves geoestadísticas municipales a las ciudades del dataset PIGOO 

In [34]:
dataset = pd.merge(dataset, nom_estandar[['CVE_MUN', 'Nombre en Dataset PIGOO']], how='left', left_on='ciudad', right_on='Nombre en Dataset PIGOO')
del(dataset['Nombre en Dataset PIGOO'])
dataset.head()

,indicador,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,ciudad,CVE_MUN
0,Tomas con servicio continuo (%),91.99,91.46,90.40,90.68,90.14,90.43,89.82,95.96,96.04,96.23,99.39,99.44,99.12,99.23,"Tecate, Baja California",02003
1,Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,"Tijuana, Baja California",02004
2,Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,78.13,98.83,98.77,99.00,97.79,97.71,97.54,97.50,NaN,"Ensenada, Baja California",02001
3,Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,"Mexicali, Baja California",02002
4,Tomas con servicio continuo (%),99.69,99.77,99.76,99.71,99.72,99.70,99.71,92.91,100.00,100.00,100.00,99.71,100.00,100.00,"Torreón, Coahuila",05035


El dataset Pigoo se separará en datasets diferentes para cada indicador. Los indicadores que incluye el dataset PIGOO son los siguientes:

In [93]:
indicadores = set(dataset['indicador'])
for i in indicadores: print('{} . {}'.format(len(i), i))

15 . Consumo (l/h/d)
23 . Eficiencia física 2 (%)
23 . Eficiencia física 1 (%)
30 . Rehabilitación de tubería  (%)
41 . Cobertura de alcantarillado reportada (%)
16 . Dotación (l/h/d)
24 . Eficiencia comercial (%)
34 . Usuarios abastecidos con pipas (%)
39 . Cobertura de agua potable reportada (%)
21 . Eficiencia global (%)
22 . Padrón de usuarios (%)
17 . Macromedición (%)
25 . Redes e instalaciones (%)
16 . Micromedición(%)
37 . Costos entre volumen producido ($/m³)
26 . Relación inversión-pib (%)
19 . Volumen tratado (%)
59 . Empleados dedicados al control de fugas (Trabajadores/fuga)
30 . Usuarios con pago a tiempo (%)
27 . Pérdidas por toma (m³/toma)
23 . Relación de trabajo (%)
23 . Relación costo - tarifa
36 . Pérdidas por longitud de red (m³/km)
37 . Horas con servicio en zonas de tandeo
42 . Rehabilitación de tomas domiciliarias  (%)
31 . Tomas con servicio continuo (%)
34 . Empleados por cada mil tómas (Núm)
23 . Eficiencia de cobro (%)
38 . Reclamaciones por cada mil tomas (

In [97]:
# Separacion de dataset por indicador
dict_indicadores = {}
for i in indicadores:
    indicador = dataset[dataset['indicador'] == i]
    dict_indicadores[i] = indicador

Con el objetivo de nombrar las hojas de cada indicador, es necesario quitar de los nombres de indicador los caracteres inválidos para nombres de hojas y reducir la longitud a menos de 31 caracteres

In [98]:
dict_indicadores['Consumo'] = dict_indicadores.pop(r'Consumo (l/h/d)')
dict_indicadores['Alcantarillado'] = dict_indicadores.pop(r'Cobertura de alcantarillado reportada (%)')
dict_indicadores['Dotacion'] = dict_indicadores.pop(r'Dotación (l/h/d)')
dict_indicadores['Abasto_por_pipas'] = dict_indicadores.pop(r'Usuarios abastecidos con pipas (%)')
dict_indicadores['Cobertura_Agua_potable'] = dict_indicadores.pop(r'Cobertura de agua potable reportada (%)')
dict_indicadores['Costos_por_volumen'] = dict_indicadores.pop(r'Costos entre volumen producido ($/m³)')
dict_indicadores['Trabajadores_por_fuga'] = dict_indicadores.pop(r'Empleados dedicados al control de fugas (Trabajadores/fuga)')
dict_indicadores['Perdidas_por_toma'] = dict_indicadores.pop(r'Pérdidas por toma (m³/toma)')
dict_indicadores['Perdidas_por_long_red'] = dict_indicadores.pop(r'Pérdidas por longitud de red (m³/km)')
dict_indicadores['Horas_tandeo'] = dict_indicadores.pop(r'Horas con servicio en zonas de tandeo')
dict_indicadores['Rehabilitacion_Tomas'] = dict_indicadores.pop(r'Rehabilitación de tomas domiciliarias  (%)')
dict_indicadores['Empleados_cada_mil_tomas'] = dict_indicadores.pop(r'Empleados por cada mil tómas (Núm)')
dict_indicadores['Reclamos_cada_mil_tomas'] = dict_indicadores.pop(r'Reclamaciones por cada mil tomas (Núm)')

In [99]:
# Estandarizacion de dataset en formato Open XML (.xlsx)
destino = r'D:\PCCS\01_Dmine\Datasets\Pigoo\Pigoo.xlsx'
writer = pd.ExcelWriter(destino)
for k, v in dict_indicadores.items():
    v.to_excel(writer, sheet_name=k.replace(r'/', '|'))
writer.close()